In [32]:
# This Python 3 environment comes with many helful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import matplotlib.pyplot as plt

/kaggle/input/nk-hackathon-dataset/test_data.parquet
/kaggle/input/nk-hackathon-dataset/train_data.parquet
/kaggle/input/nk-temp-sol-1/result_test_12(1).csv


# Loading dataset

In [33]:
data_path = '/kaggle/input/nk-hackathon-dataset'
output_path = '/kaggle/working'
temp_path = '/kaggle/temp'

In [34]:
train_df = pd.read_parquet(f'{data_path}/train_data.parquet')

# Convert timestamp from nanoseconds to datetime
train_df['datetime'] = pd.to_datetime(train_df['timestamp'])

print(train_df.columns, train_df.shape)

Index(['timestamp', 'underlying', 'expiry', 'call_iv_23500', 'call_iv_23600',
       'call_iv_23700', 'call_iv_23800', 'call_iv_23900', 'call_iv_24000',
       'call_iv_24100', 'call_iv_24200', 'call_iv_24300', 'call_iv_24400',
       'call_iv_24500', 'call_iv_24600', 'call_iv_24700', 'call_iv_24800',
       'call_iv_24900', 'call_iv_25000', 'call_iv_25100', 'call_iv_25200',
       'call_iv_25300', 'call_iv_25400', 'call_iv_25500', 'call_iv_25600',
       'call_iv_25700', 'call_iv_25800', 'call_iv_25900', 'call_iv_26000',
       'put_iv_22500', 'put_iv_22600', 'put_iv_22700', 'put_iv_22800',
       'put_iv_22900', 'put_iv_23000', 'put_iv_23100', 'put_iv_23200',
       'put_iv_23300', 'put_iv_23400', 'put_iv_23500', 'put_iv_23600',
       'put_iv_23700', 'put_iv_23800', 'put_iv_23900', 'put_iv_24000',
       'put_iv_24100', 'put_iv_24200', 'put_iv_24300', 'put_iv_24400',
       'put_iv_24500', 'put_iv_24600', 'put_iv_24700', 'put_iv_24800',
       'put_iv_24900', 'put_iv_25000', 'X0', '

In [35]:
iv_cols = [col for col in train_df.columns if 'iv' in col]

# Create a boolean mask for valid rows (all IV values between 0 and 100 inclusive)
# And remove invalid rows
valid_rows_mask = ~((train_df[iv_cols] < 0) | (train_df[iv_cols] > 100)).any(axis=1)

clean_df = train_df[valid_rows_mask].copy()

print(f"Removed {len(train_df) - len(clean_df)} invalid rows.")
train_df = clean_df
print(len(train_df))
print(clean_df.columns)

Removed 299 invalid rows.
178041
Index(['timestamp', 'underlying', 'expiry', 'call_iv_23500', 'call_iv_23600',
       'call_iv_23700', 'call_iv_23800', 'call_iv_23900', 'call_iv_24000',
       'call_iv_24100', 'call_iv_24200', 'call_iv_24300', 'call_iv_24400',
       'call_iv_24500', 'call_iv_24600', 'call_iv_24700', 'call_iv_24800',
       'call_iv_24900', 'call_iv_25000', 'call_iv_25100', 'call_iv_25200',
       'call_iv_25300', 'call_iv_25400', 'call_iv_25500', 'call_iv_25600',
       'call_iv_25700', 'call_iv_25800', 'call_iv_25900', 'call_iv_26000',
       'put_iv_22500', 'put_iv_22600', 'put_iv_22700', 'put_iv_22800',
       'put_iv_22900', 'put_iv_23000', 'put_iv_23100', 'put_iv_23200',
       'put_iv_23300', 'put_iv_23400', 'put_iv_23500', 'put_iv_23600',
       'put_iv_23700', 'put_iv_23800', 'put_iv_23900', 'put_iv_24000',
       'put_iv_24100', 'put_iv_24200', 'put_iv_24300', 'put_iv_24400',
       'put_iv_24500', 'put_iv_24600', 'put_iv_24700', 'put_iv_24800',
       'put_i

In [36]:
test_df = pd.read_parquet(f'{data_path}/test_data.parquet')
print(test_df.columns, test_df.shape)

Index(['timestamp', 'underlying', 'call_iv_24000', 'call_iv_24100',
       'call_iv_24200', 'call_iv_24300', 'call_iv_24400', 'call_iv_24500',
       'call_iv_24600', 'call_iv_24700', 'call_iv_24800', 'call_iv_24900',
       'call_iv_25000', 'call_iv_25100', 'call_iv_25200', 'call_iv_25300',
       'call_iv_25400', 'call_iv_25500', 'call_iv_25600', 'call_iv_25700',
       'call_iv_25800', 'call_iv_25900', 'call_iv_26000', 'call_iv_26100',
       'call_iv_26200', 'call_iv_26300', 'call_iv_26400', 'call_iv_26500',
       'put_iv_23000', 'put_iv_23100', 'put_iv_23200', 'put_iv_23300',
       'put_iv_23400', 'put_iv_23500', 'put_iv_23600', 'put_iv_23700',
       'put_iv_23800', 'put_iv_23900', 'put_iv_24000', 'put_iv_24100',
       'put_iv_24200', 'put_iv_24300', 'put_iv_24400', 'put_iv_24500',
       'put_iv_24600', 'put_iv_24700', 'put_iv_24800', 'put_iv_24900',
       'put_iv_25000', 'put_iv_25100', 'put_iv_25200', 'put_iv_25300',
       'put_iv_25400', 'put_iv_25500', 'X0', 'X1', 'X2',

# Feature Engineering - ATM IV

Added features like atm_strike and ATM_IV in the dataset

In [37]:
%%time
def custom_round(val):
    remainder = val % 100
    if remainder < 50:
        return val - remainder
    else:
        return val + (100 - remainder)

# Step 1: Round underlying
test_df['atm_strike'] = test_df['underlying'].apply(custom_round)

# Step 2: Generate column names
test_df['atm_call_col'] = 'call_iv_' + test_df['atm_strike'].astype(int).astype(str)
test_df['atm_put_col'] = 'put_iv_' + test_df['atm_strike'].astype(int).astype(str)

# Step 3: Compute ATM_IV as average of call and put IVs
def compute_atm_iv(row):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        call_iv = row.get(row['atm_call_col'], float('nan'))
        put_iv = row.get(row['atm_put_col'], float('nan'))
        return np.nanmean([call_iv, put_iv]) 

test_df['ATM_IV'] = test_df.apply(compute_atm_iv, axis=1)

test_df.drop(['atm_call_col', 'atm_put_col'], axis=1, inplace=True)

CPU times: user 696 ms, sys: 35 ms, total: 731 ms
Wall time: 727 ms


# Training XGBoost Regressor

In [38]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# Extract call and put IV columns
call_iv_cols = [col for col in test_df.columns if col.startswith('call_iv_')]
put_iv_cols = [col for col in test_df.columns if col.startswith('put_iv_')]
iv_cols = call_iv_cols + put_iv_cols

# Other numeric features (excluding timestamp and IV columns)
base_features = [col for col in test_df.columns if col.startswith('X')]

In [39]:
base_features

['X0',
 'X1',
 'X2',
 'X3',
 'X4',
 'X5',
 'X6',
 'X7',
 'X8',
 'X9',
 'X10',
 'X11',
 'X12',
 'X13',
 'X14',
 'X15',
 'X16',
 'X17',
 'X18',
 'X19',
 'X20',
 'X21',
 'X22',
 'X23',
 'X24',
 'X25',
 'X26',
 'X27',
 'X28',
 'X29',
 'X30',
 'X31',
 'X32',
 'X33',
 'X34',
 'X35',
 'X36',
 'X37',
 'X38',
 'X39',
 'X40',
 'X41']

In [40]:
original_iter = 2
new_iter = 10

In [41]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Create a copy of the original DataFrame
df_copy = test_df.copy()

# Initialize result dictionary
results = {}
total_cols = len(iv_cols)

for it in range(new_iter):
    print(f'\nIteration {it+1}')

    for idx, target_col in enumerate(iv_cols, 1):
        print(f'\rTraining model for: {target_col} ({idx}/{total_cols})', end='', flush=True)
                
        # Use all other IV columns + X features as input features + new features for prediction
        other_iv_cols = [col for col in iv_cols if col != target_col]
        # new_features = ['underlying']
        new_features = ['underlying', 'ATM_IV']
        feature_cols = other_iv_cols + base_features + new_features
    
        # Exclude rows where target is NaN
        df_notna = df_copy[test_df[target_col].notna()]
        df_na = df_copy[test_df[target_col].isna()]
    
        # Features and target
        X = df_notna[feature_cols]
        y = df_notna[target_col]
    
        model = XGBRegressor(
            n_estimators=200,
            max_depth=10,
            learning_rate=0.05,
            random_state=42,
            n_jobs=-1
        )
        model.fit(X, y)
    
        # Predict missing values
        if not df_na.empty:
            X_missing = df_na[feature_cols]
            y_missing_pred = model.predict(X_missing)
            df_copy.loc[df_na.index, target_col] = y_missing_pred
        else:
            y_missing_pred = np.array([])


Iteration 1
Training model for: put_iv_25500 (52/52))
Iteration 2
Training model for: put_iv_25500 (52/52))

In [42]:
cols = ['timestamp'] + iv_cols
result = df_copy[cols]
result.loc[:, iv_cols] = result.loc[:, iv_cols].round(6)
result.shape

(12065, 53)

In [43]:
result.isna().sum().sum()

0

In [44]:
result.to_csv('/kaggle/working/result.csv', index=False)

In [45]:
result.head()

,timestamp,call_iv_24000,call_iv_24100,call_iv_24200,call_iv_24300,call_iv_24400,call_iv_24500,call_iv_24600,call_iv_24700,call_iv_24800,...,put_iv_24600,put_iv_24700,put_iv_24800,put_iv_24900,put_iv_25000,put_iv_25100,put_iv_25200,put_iv_25300,put_iv_25400,put_iv_25500
0,0,0.280939,0.267103,0.258533,0.249194,0.242149,0.237596,0.232439,0.225881,0.222997,...,0.232334,0.226187,0.222644,0.227301,0.234169,0.244560,0.250422,0.257254,0.272731,0.282229
1,1,0.270276,0.269490,0.258893,0.249620,0.244143,0.238890,0.233548,0.228057,0.223973,...,0.233757,0.228209,0.223666,0.229865,0.238761,0.249402,0.258007,0.264440,0.271976,0.283739
2,2,0.256177,0.251731,0.235928,0.224953,0.214869,0.204580,0.194604,0.188133,0.182379,...,0.194612,0.188052,0.182238,0.180605,0.181346,0.185027,0.190750,0.196471,0.204691,0.211943
3,3,0.241888,0.231800,0.220505,0.208685,0.198602,0.186190,0.175848,0.166402,0.161934,...,0.175087,0.166394,0.161561,0.160259,0.164096,0.172032,0.181007,0.189043,0.196565,0.206107
4,4,0.235328,0.229805,0.222983,0.214126,0.205935,0.199265,0.192603,0.187075,0.182228,...,0.192901,0.186959,0.182131,0.177920,0.176800,0.176000,0.177700,0.178879,0.182314,0.185943
